In [10]:
!pip install numpy 1.23.5


ERROR: Could not find a version that satisfies the requirement 1.23.5 (from versions: none)
ERROR: No matching distribution found for 1.23.5


In [ ]:
# import the spotify df - the updated one
df = pd.read_csv("C:/Users/nyolc/OneDrive/Asztali gép/CS251/spotify_update_nov_2024.csv")

In [ ]:
# Check a sample of the df to make sure it all looks good


In [ ]:
# instructions to get the spotify API:

# https://developer.spotify.com/documentation/web-api

In [ ]:
CLIENT_ID = "create a client id first"
CLIENT_SECRET =  "then make sure the client secret is placed here"

In [ ]:
# use os.environ to save your client id and secret id, and delete the strings above with your API keys after running this code.
# this keeps your secret keys private in case you want to share your work


os.environ['client_id'] = CLIENT_ID
os.environ['client_secret'] = CLIENT_SECRET

In [ ]:
# retrieve  client id to check if it works

os.getenv('client_id'), os.getenv('client_secret')

('2a54b52d9b3c48c18f66a28b43782dac', '5bd18067413b4756a9dfe64bedd776e8')

In [ ]:
# Code to get your access token

# Spotify API credentials
client_id = os.getenv('client_id')
client_secret = os.getenv('client_secret')

# Spotify Accounts service endpoint
token_url = 'https://accounts.spotify.com/api/token'

# Base64 encode the client_id and client_secret
client_creds = f"{client_id}:{client_secret}"
client_creds_b64 = base64.b64encode(client_creds.encode())

# Request body parameters
token_data = {
    "grant_type": "client_credentials"
}

# Request headers
token_headers = {
    "Authorization": f"Basic {client_creds_b64.decode()}"
}

# Make the POST request to get the OAuth token
token_response = requests.post(token_url, data=token_data, headers=token_headers)

# Check if the request was successful (status code 200)
if token_response.status_code in range(200, 299):
    # Extract the access token from the response JSON
    access_token = token_response.json()['access_token']
    print("Access token success")
else:
    print("Error:", token_response.status_code)


Access token success


In [ ]:
#show access token
access_token

'BQAVLD6WhCatIGuGCrvuDMABkKdgIrJNfSBDKUQ1EOVynFeVmB2wEpNuMWNmEy5zx9Krqg16HwG8Q_puQomkpsYu6lrhJl3NbIeFK0F4v7YkkTQiKGg'

In [ ]:
# Let's search a song and get the ID
# In order to search for song features, you need the song id

search = 'Around the World'

search_url = f"https://api.spotify.com/v1/search?q={search}&type=track&access_token={access_token}"

response = requests.get(search_url)

song_id = response.json()['tracks']['items'][0]['id']

song_name = response.json()['tracks']['items'][0]['name']

artist_name = response.json()['tracks']['items'][0]['artists'][0]['name']

released_date = response.json()['tracks']['items'][0]['album']['release_date']

print('Song ID =', song_id)

Song ID = 1pKYYY0dkg23sQQXi0Q5zN


In [ ]:
# # function to get song features with the song id

def get_spotify_song_features(song_id):
    headers_dic = {"Authorization" : "Bearer " + access_token,
                   "Content-Type" : "application/x-www-form-urlencoded"}

    r = requests.get(f"https://api.spotify.com/v1/audio-features/{song_id}",
                      headers = headers_dic)
    
    result_json = json.loads(r.text)

    r_track = requests.get(f"https://api.spotify.com/v1/tracks/{song_id}", headers=headers_dic)

    track_json = json.loads(r_track.text)

    artist_id = track_json['artists'][0]['id']

    r_artist = requests.get(f"https://api.spotify.com/v1/artists/{artist_id}", headers=headers_dic)

    artist_json = json.loads(r_artist.text)

    genres = artist_json.get('genres', [])

    result_json['genres'] = genres
    
    result_json['popularity'] = track_json.get('popularity', 'N/A')

    result_json['track_number'] = track_json.get('track_number', 'N/A')

    result_json['explicit'] = track_json.get('explicit', 'N/A')

    result_json['duration_ms'] = track_json.get('duration_ms', 'N/A')

    return result_json

In [ ]:
# Call the get spotify song features function
# save the result as a variable
res = get_spotify_song_features(song_id)


In [ ]:
# print the song name, artist name, released date, and song features
print(f"Song Name: {song_name}")
print(f"Artist Name: {artist_name}")
print(f"Released Date: {released_date}")
print(f"Song Features: {res}")


Song Name: Around the World
Artist Name: Daft Punk
Released Date: 1997-01-17
Song Features: {'danceability': 0.956, 'energy': 0.795, 'key': 7, 'loudness': -5.311, 'mode': 1, 'speechiness': 0.15, 'acousticness': 0.00356, 'instrumentalness': 0.889, 'liveness': 0.0906, 'valence': 0.841, 'tempo': 121.294, 'type': 'audio_features', 'id': '1pKYYY0dkg23sQQXi0Q5zN', 'uri': 'spotify:track:1pKYYY0dkg23sQQXi0Q5zN', 'track_href': 'https://api.spotify.com/v1/tracks/1pKYYY0dkg23sQQXi0Q5zN', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1pKYYY0dkg23sQQXi0Q5zN', 'duration_ms': 429533, 'time_signature': 4, 'genres': ['electro', 'filter house', 'rock'], 'popularity': 70, 'track_number': 7, 'explicit': False}


In [ ]:
# Look at the response again. What is the data type?
# What are the keys in the response?
# What are the values of the keys?
# What are the data types of the values?
dtype= type(res)
keys = res.keys()
values = res.values()
value_types = [type(value) for value in values]
print(dtype)
print(keys)
print(values)
print(value_types)


<class 'dict'>
dict_keys(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature', 'genres', 'popularity', 'track_number', 'explicit'])
dict_values([0.956, 0.795, 7, -5.311, 1, 0.15, 0.00356, 0.889, 0.0906, 0.841, 121.294, 'audio_features', '1pKYYY0dkg23sQQXi0Q5zN', 'spotify:track:1pKYYY0dkg23sQQXi0Q5zN', 'https://api.spotify.com/v1/tracks/1pKYYY0dkg23sQQXi0Q5zN', 'https://api.spotify.com/v1/audio-analysis/1pKYYY0dkg23sQQXi0Q5zN', 429533, 4, ['electro', 'filter house', 'rock'], 70, 7, False])
[<class 'float'>, <class 'float'>, <class 'int'>, <class 'float'>, <class 'int'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'int'>, <class 'int'>, <class 'list'>, <class 'int'>, <class 'int'>, <cla

In [ ]:

# add song_name, artist_name, and released_date to the result
res['song_name'] = song_name
res['artist_name'] = artist_name
res['released_date'] = released_date


In [ ]:
# View res again. Did everything work?
print(res)

{'danceability': 0.956, 'energy': 0.795, 'key': 7, 'loudness': -5.311, 'mode': 1, 'speechiness': 0.15, 'acousticness': 0.00356, 'instrumentalness': 0.889, 'liveness': 0.0906, 'valence': 0.841, 'tempo': 121.294, 'type': 'audio_features', 'id': '1pKYYY0dkg23sQQXi0Q5zN', 'uri': 'spotify:track:1pKYYY0dkg23sQQXi0Q5zN', 'track_href': 'https://api.spotify.com/v1/tracks/1pKYYY0dkg23sQQXi0Q5zN', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1pKYYY0dkg23sQQXi0Q5zN', 'duration_ms': 429533, 'time_signature': 4, 'genres': ['electro', 'filter house', 'rock'], 'popularity': 70, 'track_number': 7, 'explicit': False, 'song_name': 'Around the World', 'artist_name': 'Daft Punk', 'released_date': '1997-01-17'}


In [ ]:
# Make a one row dataframe of the song info from the above dictionary
df = pd.DataFrame([res])

# View the dataframe to make sure it worked
print(df)


   danceability  energy  key  loudness  mode  speechiness  acousticness  \
0         0.956   0.795    7    -5.311     1         0.15       0.00356   

   instrumentalness  liveness  valence    tempo            type  \
0             0.889    0.0906    0.841  121.294  audio_features   

                       id                                   uri  \
0  1pKYYY0dkg23sQQXi0Q5zN  spotify:track:1pKYYY0dkg23sQQXi0Q5zN   

                                          track_href  \
0  https://api.spotify.com/v1/tracks/1pKYYY0dkg23...   

                                        analysis_url  duration_ms  \
0  https://api.spotify.com/v1/audio-analysis/1pKY...       429533   

   time_signature                         genres  popularity  track_number  \
0               4  [electro, filter house, rock]          70             7   

   explicit         song_name artist_name released_date  
0     False  Around the World   Daft Punk    1997-01-17  


In [ ]:
# Try it with a different song - can you search the song, find the ID, search the features, and create a one row dataframe called df2?
search2 = 'Time'


In [ ]:
# Search for a different song and get the ID 
song2='Time'
search_url2 = f"https://api.spotify.com/v1/search?q={search2}&type=track&access_token={access_token}"


In [ ]:
# Call the get song features function using the song id printed above.
# Save the result as res2
res2 = requests.get(search_url2)
song_id2 = res2.json()['tracks']['items'][0]['id']
res2 = get_spotify_song_features(song_id2)
res2


{'danceability': 0.769,
 'energy': 0.722,
 'key': 11,
 'loudness': -5.485,
 'mode': 0,
 'speechiness': 0.0507,
 'acousticness': 0.0584,
 'instrumentalness': 2.56e-06,
 'liveness': 0.111,
 'valence': 0.57,
 'tempo': 119.973,
 'type': 'audio_features',
 'id': '1Es7AUAhQvapIcoh3qMKDL',
 'uri': 'spotify:track:1Es7AUAhQvapIcoh3qMKDL',
 'track_href': 'https://api.spotify.com/v1/tracks/1Es7AUAhQvapIcoh3qMKDL',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1Es7AUAhQvapIcoh3qMKDL',
 'duration_ms': 256000,
 'time_signature': 4,
 'genres': ['canadian contemporary r&b', 'canadian pop', 'pop'],
 'popularity': 92,
 'track_number': 1,
 'explicit': True}

In [ ]:
# Add the track name, artist name, and released date to the res2 dictionary
res2['song_name'] = song_name
res2['artist_name'] = artist_name
res2['released_date'] = released_date




# Look at the dictionary to make sure it all worked
print(res2)


{'danceability': 0.769, 'energy': 0.722, 'key': 11, 'loudness': -5.485, 'mode': 0, 'speechiness': 0.0507, 'acousticness': 0.0584, 'instrumentalness': 2.56e-06, 'liveness': 0.111, 'valence': 0.57, 'tempo': 119.973, 'type': 'audio_features', 'id': '1Es7AUAhQvapIcoh3qMKDL', 'uri': 'spotify:track:1Es7AUAhQvapIcoh3qMKDL', 'track_href': 'https://api.spotify.com/v1/tracks/1Es7AUAhQvapIcoh3qMKDL', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1Es7AUAhQvapIcoh3qMKDL', 'duration_ms': 256000, 'time_signature': 4, 'genres': ['canadian contemporary r&b', 'canadian pop', 'pop'], 'popularity': 92, 'track_number': 1, 'explicit': True, 'song_name': 'Around the World', 'artist_name': 'Daft Punk', 'released_date': '1997-01-17'}


In [ ]:
# Make a one row dataframe (with a different name from the first one you made) of the song info from the above dictionary
df2 = pd.DataFrame([res2])




# view df2 to make sure it all worked


In [ ]:
# Put the two dataframes together, one on top of the other
df_all = pd.concat([df, df2])


# View the dataframe to make sure it worked
print(df_all)

   danceability  energy  key  loudness  mode  speechiness  acousticness  \
0         0.956   0.795    7    -5.311     1       0.1500       0.00356   
0         0.769   0.722   11    -5.485     0       0.0507       0.05840   

   instrumentalness  liveness  valence    tempo            type  \
0          0.889000    0.0906    0.841  121.294  audio_features   
0          0.000003    0.1110    0.570  119.973  audio_features   

                       id                                   uri  \
0  1pKYYY0dkg23sQQXi0Q5zN  spotify:track:1pKYYY0dkg23sQQXi0Q5zN   
0  1Es7AUAhQvapIcoh3qMKDL  spotify:track:1Es7AUAhQvapIcoh3qMKDL   

                                          track_href  \
0  https://api.spotify.com/v1/tracks/1pKYYY0dkg23...   
0  https://api.spotify.com/v1/tracks/1Es7AUAhQvap...   

                                        analysis_url  duration_ms  \
0  https://api.spotify.com/v1/audio-analysis/1pKY...       429533   
0  https://api.spotify.com/v1/audio-analysis/1Es7...       2560

In [ ]:
df_all=pd.concat([df, df2])

In [ ]:
df_all


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genres,popularity,track_number,explicit,song_name,artist_name,released_date
0,0.956,0.795,7,-5.311,1,0.1500,0.00356,0.889000,0.0906,0.841,121.294,audio_features,1pKYYY0dkg23sQQXi0Q5zN,spotify:track:1pKYYY0dkg23sQQXi0Q5zN,https://api.spotify.com/v1/tracks/1pKYYY0dkg23...,https://api.spotify.com/v1/audio-analysis/1pKY...,429533,4,"[electro, filter house, rock]",70,7,False,Around the World,Daft Punk,1997-01-17
0,0.769,0.722,11,-5.485,0,0.0507,0.05840,0.000003,0.1110,0.570,119.973,audio_features,1Es7AUAhQvapIcoh3qMKDL,spotify:track:1Es7AUAhQvapIcoh3qMKDL,https://api.spotify.com/v1/tracks/1Es7AUAhQvap...,https://api.spotify.com/v1/audio-analysis/1Es7...,256000,4,"[canadian contemporary r&b, canadian pop, pop]",92,1,True,Around the World,Daft Punk,1997-01-17


In [ ]:
def make_song_function(track_name, df_all, access_token=access_token):
    search_url = f"https://api.spotify.com/v1/search?q=track:\"{track_name}\"&type=track&limit=20&access_token={access_token}"
    response = requests.get(search_url)
    tracks = response.json()['tracks']['items']
    
    exact_matches = [t for t in tracks if track_name.lower() == t['name'].lower()]
    partial_matches = [t for t in tracks if track_name.lower() in t['name'].lower() and t not in exact_matches]
    
    filtered_tracks = exact_matches + partial_matches
    filtered_tracks.sort(key=lambda x: x['popularity'], reverse=True)
    filtered_tracks = filtered_tracks[:5]  # Limit to top 5 most popular matches

    if not filtered_tracks:
        print("No matches found. Try a different search term.")
        return df

    print("\nAvailable tracks:")
    print("-" * 80)
    for i, track in enumerate(filtered_tracks, 1):
        artists = ", ".join([artist['name'] for artist in track['artists']])
        print(f"{i}. '{track['name']}' by {artists}")
        print(f"   Popularity: {track['popularity']}")
        print(f"   Album: {track['album']['name']}")
        print(f"   Release Date: {track['album']['release_date']}")
        print("-" * 80)

    while True:
        try:
            selection = int(input("Select a track number (1-5) or 0 to cancel: "))
            if selection == 0:
                return df
            if 1 <= selection <= len(filtered_tracks):
                break
            print(f"Please enter a number between 1 and {len(filtered_tracks)}")
        except ValueError:
            print("Please enter a valid number")

    selected_track = filtered_tracks[selection-1]

    res = get_spotify_song_features(selected_track['id'])

    res['song_name'] = selected_track['name']
    res['artist_name'] = selected_track['artists'][0]['name']
    res['released_date'] = selected_track['album']['release_date']

    if df.empty or res['song_name'] not in df['song_name'].values:
        temp_df = pd.DataFrame([res])
        updated_df = pd.concat([df, temp_df], ignore_index=True)
        return updated_df
    else:
        print(f"'{res['song_name']}' is already in the DataFrame!")
        return df

df = pd.DataFrame()

df = make_song_function("505", df)
display(df)


Available tracks:
--------------------------------------------------------------------------------
1. '505' by Arctic Monkeys
   Popularity: 79
   Album: Favourite Worst Nightmare (Standard Version)
   Release Date: 2007-04-24
--------------------------------------------------------------------------------
2. '505 - Sped Up' by Speedy Jack
   Popularity: 49
   Album: 505 (Sped Up)
   Release Date: 2023-07-25
--------------------------------------------------------------------------------
3. '505 hoodtrap' by Sims
   Popularity: 48
   Album: 505 hoodtrap
   Release Date: 2024-10-19
--------------------------------------------------------------------------------
4. '5050' by MANBO, HIEUTHUHAI, HURRYKNG
   Popularity: 48
   Album: 5050
   Release Date: 2020-10-09
--------------------------------------------------------------------------------
5. '505 sped up' by Do you know Calliandra, Ryugan, missvalentine
   Popularity: 45
   Album: 505 sped up
   Release Date: 2022-02-24
-------------

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genres,popularity,track_number,explicit,song_name,artist_name,released_date
0,0.52,0.852,0,-5.866,1,0.0543,0.00237,0.000058,0.0733,0.234,140.267,audio_features,58ge6dfP91o9oXMzq3XkIS,spotify:track:58ge6dfP91o9oXMzq3XkIS,https://api.spotify.com/v1/tracks/58ge6dfP91o9...,https://api.spotify.com/v1/audio-analysis/58ge...,253586,4,"[garage rock, modern rock, permanent wave, roc...",79,12,False,505,Arctic Monkeys,2007-04-24


In [ ]:
import requests
import pandas as pd
from datetime import datetime
import base64

# Set up Spotify API credentials
CLIENT_ID = "client id here"
CLIENT_SECRET = "secret paired to client id"

# Function to get Spotify API access token
def get_spotify_token(client_id, client_secret):
    token_url = 'https://accounts.spotify.com/api/token'
    client_creds = f"{client_id}:{client_secret}"
    client_creds_b64 = base64.b64encode(client_creds.encode()).decode()

    # Token request parameters and headers
    token_data = {"grant_type": "client_credentials"}
    token_headers = {"Authorization": f"Basic {client_creds_b64}"}

    # Send token request
    response = requests.post(token_url, data=token_data, headers=token_headers)
    response.raise_for_status()
    return response.json().get("access_token")

# Function to search for a track and get its ID
def search_track(track_name, access_token):
    search_url = f"https://api.spotify.com/v1/search?q={track_name}&type=track"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(search_url, headers=headers)
    response.raise_for_status()
    results = response.json()
    
    # Extract the first track's ID if found
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        return track['id'], track['name'], track['artists'][0]['name'], track['album']['release_date']
    return None, None, None, None

# Function to get song features by ID
def get_spotify_song_features(song_id, access_token):
    url = f"https://api.spotify.com/v1/audio-features/{song_id}"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

# Function to add song data to DataFrame
def make_song_function(track_name, df_all, access_token):
    # Search for the track to get the ID and metadata
    song_id, song_name, artist_name, release_date = search_track(track_name, access_token)
    if not song_id:
        print(f"Track '{track_name}' not found.")
        return df_all

    # Get song features from Spotify
    song_features = get_spotify_song_features(song_id, access_token)
    
    # Add additional metadata
    song_features['track_name'] = song_name
    song_features['artist_name'] = artist_name
    song_features['release_date'] = release_date
    
    # Convert release_date to datetime format and extract additional date info
    release_date_dt = datetime.strptime(release_date, "%Y-%m-%d")
    song_features['released_year'] = release_date_dt.year
    song_features['released_month'] = release_date_dt.month
    song_features['released_day_name'] = release_date_dt.strftime('%A')
    
    # Convert specific features to percentages
    for col in ['danceability', 'valence', 'energy', 'acousticness', 'instrumentalness', 'speechiness', 'liveness']:
        if col in song_features:
            song_features[col] *= 100  # Convert to percentage
    
    # Remove unwanted columns
    for col in ['id', 'uri', 'track_href', 'analysis_url']:
        song_features.pop(col, None)
    
    # Append song data to the DataFrame
    df_new_song = pd.DataFrame([song_features])
    return pd.concat([df_all, df_new_song], ignore_index=True)

# Initialize an empty DataFrame for the songs
df_all = pd.DataFrame()

# Get access token for Spotify API
access_token = get_spotify_token(CLIENT_ID, CLIENT_SECRET)

# Define the list of songs to add
songs = ['Time', 'Around the World', '505', 'Lose Yourself to Dance', 'Get Lucky']

# Add each song to the DataFrame
for song in songs:
    df_all = make_song_function(song, df_all, access_token)

# Rename and create necessary columns
df_all.rename(columns={'track_name': 'song_name'}, inplace=True)

# Save DataFrame as a CSV
csv_filename = "spotify_songs.csv"
df_all.to_csv(csv_filename, index=False)

# Load and display the saved CSV file to confirm
df_loaded = pd.read_csv(csv_filename)
df_loaded.head()


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,duration_ms,time_signature,song_name,artist_name,release_date,released_year,released_month,released_day_name
0,76.9,72.2,11,-5.485,0,5.07,5.840,0.000256,11.10,57.0,119.973,audio_features,256000,4,Timeless (with Playboi Carti),The Weeknd,2024-09-27,2024,9,Friday
1,95.6,79.5,7,-5.311,1,15.00,0.356,88.900000,9.06,84.1,121.294,audio_features,429533,4,Around the World,Daft Punk,1997-01-17,1997,1,Friday
2,52.0,85.2,0,-5.866,1,5.43,0.237,0.005790,7.33,23.4,140.267,audio_features,253587,4,505,Arctic Monkeys,2007-04-24,2007,4,Tuesday
3,83.2,65.9,10,-7.828,0,5.70,8.390,0.114000,7.53,67.4,100.163,audio_features,353893,4,Lose Yourself to Dance (feat. Pharrell Williams),Daft Punk,2013-05-20,2013,5,Monday
4,81.0,79.3,6,-9.404,0,4.03,3.780,0.000177,7.20,86.3,116.050,audio_features,369627,4,Get Lucky (feat. Pharrell Williams and Nile Ro...,Daft Punk,2013-05-20,2013,5,Monday


In [ ]:
def make_song_function(songs_to_search):
    df = pd.DataFrame()
    i = 0  
    
    while i < len(songs_to_search):
        track_name = songs_to_search[i]
        search_url = f"https://api.spotify.com/v1/search?q=track:\"{track_name}\"&type=track&limit=20&access_token={access_token}"
        response = requests.get(search_url)
        tracks = response.json()['tracks']['items']
        
        exact_matches = [t for t in tracks if track_name.lower() == t['name'].lower()]
        partial_matches = [t for t in tracks if track_name.lower() in t['name'].lower() and t not in exact_matches]
        
        filtered_tracks = exact_matches + partial_matches
        filtered_tracks.sort(key=lambda x: x['popularity'], reverse=True)
        filtered_tracks = filtered_tracks[:5]

        if not filtered_tracks:
            print(f"\nNo matches found for '{track_name}'. Moving to next song.")
            i += 1
            continue

        print(f"\nSearching for: {track_name} ({i+1} of {len(songs_to_search)})")
        print("\nAvailable tracks:")
        print("-" * 80)
        for idx, track in enumerate(filtered_tracks, 1):
            artists = ", ".join([artist['name'] for artist in track['artists']])
            print(f"{idx}. '{track['name']}' by {artists}")
            print(f"   Popularity: {track['popularity']}")
            print(f"   Album: {track['album']['name']}")
            print(f"   Release Date: {track['album']['release_date']}")
            print("-" * 80)

        while True:
            try:
                selection = int(input(f"Select a track number (1-5) for '{track_name}' or 0 to skip: "))
                if selection == 0:
                    print(f"Skipping '{track_name}'")
                    i += 1  # Move to next song
                    break
                if 1 <= selection <= len(filtered_tracks):
                    selected_track = filtered_tracks[selection-1]
                    res = get_spotify_song_features(selected_track['id'])
                    res['song_name'] = selected_track['name']
                    res['artist_name'] = selected_track['artists'][0]['name']
                    res['released_date'] = selected_track['album']['release_date']

                    if df.empty or res['song_name'] not in df['song_name'].values:
                        temp_df = pd.DataFrame([res])
                        df = pd.concat([df, temp_df], ignore_index=True)
                        print(f"\nAdded '{res['song_name']}' to the DataFrame!")
                        print("\nCurrent DataFrame:")
                        display(df)
                        i += 1  # Move to next song only after successful addition
                        break
                    else:
                        print(f"\n'{res['song_name']}' is already in the DataFrame!")
                        continue
                        
                print(f"Please enter a number between 1 and {len(filtered_tracks)}")
            except ValueError:
                print("Please enter a valid number")
    
    return df
display(df_all)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,duration_ms,time_signature,song_name,artist_name,release_date,released_year,released_month,released_day_name,track_name
0,76.9,72.2,11,-5.485,0,5.07,5.840,0.000256,11.10,57.0,119.973,audio_features,256000,4,Timeless (with Playboi Carti),The Weeknd,2024-09-27,2024,9,Friday,NaN
1,95.6,79.5,7,-5.311,1,15.00,0.356,88.900000,9.06,84.1,121.294,audio_features,429533,4,Around the World,Daft Punk,1997-01-17,1997,1,Friday,NaN
2,52.0,85.2,0,-5.866,1,5.43,0.237,0.005790,7.33,23.4,140.267,audio_features,253587,4,505,Arctic Monkeys,2007-04-24,2007,4,Tuesday,NaN
3,83.2,65.9,10,-7.828,0,5.70,8.390,0.114000,7.53,67.4,100.163,audio_features,353893,4,Lose Yourself to Dance (feat. Pharrell Williams),Daft Punk,2013-05-20,2013,5,Monday,NaN
4,81.0,79.3,6,-9.404,0,4.03,3.780,0.000177,7.20,86.3,116.050,audio_features,369627,4,Get Lucky (feat. Pharrell Williams and Nile Ro...,Daft Punk,2013-05-20,2013,5,Monday,NaN
5,76.9,72.2,11,-5.485,0,5.07,5.840,0.000256,11.10,57.0,119.973,audio_features,256000,4,NaN,The Weeknd,2024-09-27,2024,9,Friday,Timeless (with Playboi Carti)
6,95.6,79.5,7,-5.311,1,15.00,0.356,88.900000,9.06,84.1,121.294,audio_features,429533,4,NaN,Daft Punk,1997-01-17,1997,1,Friday,Around the World
7,52.0,85.2,0,-5.866,1,5.43,0.237,0.005790,7.33,23.4,140.267,audio_features,253587,4,NaN,Arctic Monkeys,2007-04-24,2007,4,Tuesday,505
8,83.2,65.9,10,-7.828,0,5.70,8.390,0.114000,7.53,67.4,100.163,audio_features,353893,4,NaN,Daft Punk,2013-05-20,2013,5,Monday,Lose Yourself to Dance (feat. Pharrell Williams)
9,81.0,79.3,6,-9.404,0,4.03,3.780,0.000177,7.20,86.3,116.050,audio_features,369627,4,NaN,Daft Punk,2013-05-20,2013,5,Monday,Get Lucky (feat. Pharrell Williams and Nile Ro...


In [ ]:
# reate the oneline dictionary for the songs in the list and add them to the dataframe
#rename and create the necessary columns
#save the dataframe as a csv file
#check the csv file to make sure it all worked
#make sure you have the csv file in the same folder as this notebook
#create track_name, artist_name, and released_date columns
#concatenate the two dataframes

# Define the list of songs to add
songs = ['Time', 'Around the World', '505', 'Lose Yourself to Dance', 'Get Lucky']


In [ ]:
# View the datframe. Did the new songs and their features get added correctly?
print(df_all)



    danceability  energy  key  loudness  mode  speechiness  acousticness  \
0           76.9    72.2   11    -5.485     0         5.07         5.840   
1           95.6    79.5    7    -5.311     1        15.00         0.356   
2           52.0    85.2    0    -5.866     1         5.43         0.237   
3           83.2    65.9   10    -7.828     0         5.70         8.390   
4           81.0    79.3    6    -9.404     0         4.03         3.780   
5           76.9    72.2   11    -5.485     0         5.07         5.840   
6           95.6    79.5    7    -5.311     1        15.00         0.356   
7           52.0    85.2    0    -5.866     1         5.43         0.237   
8           83.2    65.9   10    -7.828     0         5.70         8.390   
9           81.0    79.3    6    -9.404     0         4.03         3.780   
10          61.3    69.7    2    -8.618     1        13.30         1.940   
11          64.4    66.4    9    -8.398     1         3.32         4.800   
12          

In [ ]:
# Check the values in your new dataframe and compare them to the original. 
# What should you do to change the feature values into percents?
df_all.head()

# Convert specific features to percentages
for col in ['danceability', 'valence', 'energy', 'acousticness', 'instrumentalness', 'speechiness', 'liveness']:
    if col in df_all:
        df_all[col] *= 100  # Convert to percentage
        


In [ ]:
# Make the feature values into %s
df_all.head()


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,duration_ms,time_signature,song_name,artist_name,release_date,released_year,released_month,released_day_name,track_name
0,769000.0,722000.0,11,-5.485,0,50700.0,58400.0,2.56,111000.0,570000.0,119.973,audio_features,256000,4,Timeless (with Playboi Carti),The Weeknd,2024-09-27,2024,9,Friday,NaN
1,956000.0,795000.0,7,-5.311,1,150000.0,3560.0,889000.00,90600.0,841000.0,121.294,audio_features,429533,4,Around the World,Daft Punk,1997-01-17,1997,1,Friday,NaN
2,520000.0,852000.0,0,-5.866,1,54300.0,2370.0,57.90,73300.0,234000.0,140.267,audio_features,253587,4,505,Arctic Monkeys,2007-04-24,2007,4,Tuesday,NaN
3,832000.0,659000.0,10,-7.828,0,57000.0,83900.0,1140.00,75300.0,674000.0,100.163,audio_features,353893,4,Lose Yourself to Dance (feat. Pharrell Williams),Daft Punk,2013-05-20,2013,5,Monday,NaN
4,810000.0,793000.0,6,-9.404,0,40300.0,37800.0,1.77,72000.0,863000.0,116.050,audio_features,369627,4,Get Lucky (feat. Pharrell Williams and Nile Ro...,Daft Punk,2013-05-20,2013,5,Monday,NaN


In [ ]:
# Check out the key values in the API dataframe - they are numbers, not letters
# We can convert these to letters for easier reading
# Create a dictionary to convert the key values to letters
key_dict = {0: 'C', 1: 'C#/Db', 2: 'D', 3: 'D#/Eb', 4: 'E', 5: 'F', 6: 'F#/Gb', 7: 'G', 8: 'G#/Ab', 9: 'A', 10: 'A#/Bb', 11: 'B'}


# find info about how to convert key and mode here: https://developer.spotify.com/documentation/web-api/reference/get-audio-features
# Convert the key and mode values to letters
df_all['key'] = df_all['key'].map(key_dict)
df_all['mode'] = df_all['mode'].map({0: 'Minor', 1: 'Major'})



In [ ]:
# Look uo how to translate key numbers to the actual letter value of the key.
# Make a map - key numbers to key letters

# Check the dataframe to make sure it worked
keys_map = {}
for i in range(12):
    keys_map[i] = key_dict[i]
keys_map


{0: 'C',
 1: 'C#/Db',
 2: 'D',
 3: 'D#/Eb',
 4: 'E',
 5: 'F',
 6: 'F#/Gb',
 7: 'G',
 8: 'G#/Ab',
 9: 'A',
 10: 'A#/Bb',
 11: 'B'}

In [ ]:
# Use the map to update the key column in the API dataframe

df_all['key'] = df_all['key'].map(keys_map)
df_all.head()



,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,duration_ms,time_signature,song_name,artist_name,release_date,released_year,released_month,released_day_name,track_name
0,769000.0,722000.0,NaN,-5.485,Minor,50700.0,58400.0,2.56,111000.0,570000.0,119.973,audio_features,256000,4,Timeless (with Playboi Carti),The Weeknd,2024-09-27,2024,9,Friday,NaN
1,956000.0,795000.0,NaN,-5.311,Major,150000.0,3560.0,889000.00,90600.0,841000.0,121.294,audio_features,429533,4,Around the World,Daft Punk,1997-01-17,1997,1,Friday,NaN
2,520000.0,852000.0,NaN,-5.866,Major,54300.0,2370.0,57.90,73300.0,234000.0,140.267,audio_features,253587,4,505,Arctic Monkeys,2007-04-24,2007,4,Tuesday,NaN
3,832000.0,659000.0,NaN,-7.828,Minor,57000.0,83900.0,1140.00,75300.0,674000.0,100.163,audio_features,353893,4,Lose Yourself to Dance (feat. Pharrell Williams),Daft Punk,2013-05-20,2013,5,Monday,NaN
4,810000.0,793000.0,NaN,-9.404,Minor,40300.0,37800.0,1.77,72000.0,863000.0,116.050,audio_features,369627,4,Get Lucky (feat. Pharrell Williams and Nile Ro...,Daft Punk,2013-05-20,2013,5,Monday,NaN


In [ ]:
# Check a row of your dataframe or the unique key values to make sure it worked.
df_all['key'].unique()




array([nan], dtype=object)

In [ ]:
# The mode is also in numbers, rather than major and minor.
# Can we fix this?
# Create a map for the mode values
mode_map = {0: 'Minor', 1: 'Major'}


In [ ]:
# Check the mode values in your dataframe
df_all['mode'].unique()



In [ ]:
# Make a mapping and update the mode column
df_all['mode'] = df_all['mode'].map(mode_map)
df_all.head()
    


In [ ]:
# make year, month, day, and day of week column from the released_date column
# Convert release_date to datetime format and extract additional date info
df_all['release_date'] = pd.to_datetime(df_all['release_date'])
df_all['released_year'] = df_all['release_date'].dt.year
df_all['released_month'] = df_all['release_date'].dt.month
df_all['released_day'] = df_all['release_date'].dt.day
df_all['released_day_name'] = df_all['release_date'].dt.day_name()
df_all.head()


In [ ]:
# Split the artist(s) column into main artist and supporting artists
# Split the artist column into main artist and supporting artists
# Split the artist column into main artist and supporting artists

#can only split the artist column if it is a string, so we need to convert it to a string first
df_all['artist_name'] = df_all['artist_name'].astype(str)
df_all['artist_name'] = df_all['artist_name'].str.split(', ')

df_all['main_artist'] = df_all['artist_name'].str[0]
df_all['supporting_artists'] = df_all['artist_name'].str[1:].apply(lambda x: ', '.join(x) if x else None)
df_all.drop(columns='artist_name', inplace=True)
df_all.head()




,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,duration_ms,time_signature,song_name,release_date,released_year,released_month,released_day_name,track_name,main_artist,supporting_artists
0,769000.0,722000.0,NaN,-5.485,Minor,50700.0,58400.0,2.56,111000.0,570000.0,119.973,audio_features,256000,4,Timeless (with Playboi Carti),2024-09-27,2024,9,Friday,NaN,The Weeknd,None
1,956000.0,795000.0,NaN,-5.311,Major,150000.0,3560.0,889000.00,90600.0,841000.0,121.294,audio_features,429533,4,Around the World,1997-01-17,1997,1,Friday,NaN,Daft Punk,None
2,520000.0,852000.0,NaN,-5.866,Major,54300.0,2370.0,57.90,73300.0,234000.0,140.267,audio_features,253587,4,505,2007-04-24,2007,4,Tuesday,NaN,Arctic Monkeys,None
3,832000.0,659000.0,NaN,-7.828,Minor,57000.0,83900.0,1140.00,75300.0,674000.0,100.163,audio_features,353893,4,Lose Yourself to Dance (feat. Pharrell Williams),2013-05-20,2013,5,Monday,NaN,Daft Punk,None
4,810000.0,793000.0,NaN,-9.404,Minor,40300.0,37800.0,1.77,72000.0,863000.0,116.050,audio_features,369627,4,Get Lucky (feat. Pharrell Williams and Nile Ro...,2013-05-20,2013,5,Monday,NaN,Daft Punk,None


In [ ]:
#create artist count column
# with the main_artist and supporting_artists columns
df_all['artist_count'] = df_all['supporting_artists'].str.count(',') + 1
df_all.head()



,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,duration_ms,time_signature,song_name,release_date,released_year,released_month,released_day_name,track_name,main_artist,supporting_artists,artist_count
0,769000.0,722000.0,NaN,-5.485,Minor,50700.0,58400.0,2.56,111000.0,570000.0,119.973,audio_features,256000,4,Timeless (with Playboi Carti),2024-09-27,2024,9,Friday,NaN,The Weeknd,None,NaN
1,956000.0,795000.0,NaN,-5.311,Major,150000.0,3560.0,889000.00,90600.0,841000.0,121.294,audio_features,429533,4,Around the World,1997-01-17,1997,1,Friday,NaN,Daft Punk,None,NaN
2,520000.0,852000.0,NaN,-5.866,Major,54300.0,2370.0,57.90,73300.0,234000.0,140.267,audio_features,253587,4,505,2007-04-24,2007,4,Tuesday,NaN,Arctic Monkeys,None,NaN
3,832000.0,659000.0,NaN,-7.828,Minor,57000.0,83900.0,1140.00,75300.0,674000.0,100.163,audio_features,353893,4,Lose Yourself to Dance (feat. Pharrell Williams),2013-05-20,2013,5,Monday,NaN,Daft Punk,None,NaN
4,810000.0,793000.0,NaN,-9.404,Minor,40300.0,37800.0,1.77,72000.0,863000.0,116.050,audio_features,369627,4,Get Lucky (feat. Pharrell Williams and Nile Ro...,2013-05-20,2013,5,Monday,NaN,Daft Punk,None,NaN


In [ ]:
#show artist count column
df_all['artist_count']


0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6     NaN
7     NaN
8     NaN
9     NaN
10    NaN
11    NaN
12    NaN
13    NaN
14    NaN
15    NaN
16    NaN
17    NaN
18    NaN
19    NaN
20    NaN
21    NaN
22    NaN
23    NaN
24    NaN
25    NaN
26    NaN
27    NaN
28    NaN
29    NaN
30    NaN
31    NaN
32    NaN
33    NaN
34    NaN
35    NaN
36    NaN
37    NaN
38    NaN
39    NaN
40    NaN
41    NaN
42    NaN
43    NaN
44    NaN
45    NaN
46    NaN
47    NaN
48    NaN
49    NaN
50    NaN
Name: artist_count, dtype: object

In [ ]:
# Add season column
# Add a season column to the dataframe
# Use the month column to determine the season
# Winter = 12, 1, 2
# Spring = 3, 4, 5
# Summer = 6, 7, 8
# Fall = 9, 10, 11
# Create a dictionary to map the month to the season
season_map = {12: 'Winter', 1: 'Winter', 2: 'Winter',
              3: 'Spring', 4: 'Spring', 5: 'Spring',
              6: 'Summer', 7: 'Summer', 8: 'Summer',
              9: 'Fall', 10: 'Fall', 11: 'Fall'}



In [ ]:
# Check a row of the df to make sure it all worked
df_all['season'] = df_all['released_month'].map(season_map)
df_all.head()


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,duration_ms,time_signature,song_name,release_date,released_year,released_month,released_day_name,track_name,main_artist,supporting_artists,artist_count,season
0,769000.0,722000.0,NaN,-5.485,Minor,50700.0,58400.0,2.56,111000.0,570000.0,119.973,audio_features,256000,4,Timeless (with Playboi Carti),2024-09-27,2024,9,Friday,NaN,The Weeknd,None,NaN,Fall
1,956000.0,795000.0,NaN,-5.311,Major,150000.0,3560.0,889000.00,90600.0,841000.0,121.294,audio_features,429533,4,Around the World,1997-01-17,1997,1,Friday,NaN,Daft Punk,None,NaN,Winter
2,520000.0,852000.0,NaN,-5.866,Major,54300.0,2370.0,57.90,73300.0,234000.0,140.267,audio_features,253587,4,505,2007-04-24,2007,4,Tuesday,NaN,Arctic Monkeys,None,NaN,Spring
3,832000.0,659000.0,NaN,-7.828,Minor,57000.0,83900.0,1140.00,75300.0,674000.0,100.163,audio_features,353893,4,Lose Yourself to Dance (feat. Pharrell Williams),2013-05-20,2013,5,Monday,NaN,Daft Punk,None,NaN,Spring
4,810000.0,793000.0,NaN,-9.404,Minor,40300.0,37800.0,1.77,72000.0,863000.0,116.050,audio_features,369627,4,Get Lucky (feat. Pharrell Williams and Nile Ro...,2013-05-20,2013,5,Monday,NaN,Daft Punk,None,NaN,Spring


In [ ]:
# Choose which columns to keep from your API dataframe
# Drop the columns you don't want
df_all.columns


Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'duration_ms', 'time_signature', 'song_name', 'release_date',
       'released_year', 'released_month', 'released_day_name', 'track_name',
       'main_artist', 'supporting_artists', 'artist_count', 'season'],
      dtype='object')

In [ ]:
# create a function to clean the new API dataframe - everything we did above, but in a function
def clean_spotify_df(df):
    # Convert specific features to percentages
    for col in ['danceability', 'valence', 'energy', 'acousticness', 'instrumentalness', 'speechiness', 'liveness']:
        if col in df:
            df[col] *= 100  # Convert to percentage

    # Convert key and mode values to letters
    key_dict = {0: 'C', 1: 'C#/Db', 2: 'D', 3: 'D#/Eb', 4: 'E', 5: 'F', 6: 'F#/Gb', 7: 'G', 8: 'G#/Ab', 9: 'A', 10: 'A#/Bb', 11: 'B'}
    df['key'] = df['key'].map(key_dict)
    df['mode'] = df['mode'].map({0: 'Minor', 1: 'Major'})

    # Convert release_date to datetime format and extract additional date info
    df['release_date'] = pd.to_datetime(df['release_date'])
    df['released_year'] = df['release_date'].dt.year
    df['released_month'] = df['release_date'].dt.month
    df['released_day'] = df['release_date'].dt.day
    df['released_day_name'] = df['release_date'].dt.day_name()

    # Split the artist column into main artist and supporting artists
    df['artist_name'] = df['artist_name'].astype(str)
    df['artist_name'] = df['artist_name'].str.split(', ')
    df['main_artist'] = df['artist_name'].str[0]
    df['supporting_artists'] = df['artist_name'].str[1:].apply(lambda x: ', '.join(x) if x else None)
    df.drop(columns='artist_name', inplace=True)

    # Create artist count column
    df['artist_count'] = df['supporting_artists'].str.count(',') + 1

    # Add season column
    season_map = {12: 'Winter', 1: 'Winter', 2: 'Winter',
                  3: 'Spring', 4: 'Spring', 5: 'Spring',
                  6: 'Summer', 7: 'Summer', 8: 'Summer',
                  9: 'Fall', 10: 'Fall', 11: 'Fall'}
    df['season'] = df['released_month'].map(season_map)
    